In [1]:
import os

download_name = "worldcities.csv.bz2"
if not os.path.exists(download_name):
    import requests
    response = requests.get(f"https://raw.githubusercontent.com/bzitko/nlp_repo/main/lectures/p03/{download_name}")
    with open(download_name, "wb") as fp:
        fp.write(response.content)
    response.close()

name = "worldcities.csv"
if not os.path.exists(name):
    import bz2
    with open(download_name, 'rb') as bzf, open(name, 'wb') as fp:
        fp.write(bz2.decompress(bzf.read()))        

In [2]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [3]:
args = Namespace(
    raw_dataset_csv="worldcities.csv",
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="worldcities_with_splits.csv",
    seed=1337
)

### Read Dataset

👍 Read raw dataset's csv file into pandas DataFrame. Use only `city` and `country` columns.

👍 Count how many datapoints are in each class.

### Split to TRAIN, VAL, TEST 

👍 For each country that has equal or more than 7 cities make train, val and test split. Use percentages defined in args do determine how many datapoints will be for train, val and test.
Append dataframe with split column.

### Save dataset

👍 Save DataFrame with header into comma separated file defined by arguments.